In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    verbose=True,
)


def load_memory(_):
    return chat_memory.load_memory_variables({})['chat_history']

In [3]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
examples = [
    {
        "movie": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "갓 파더",
        "answer": "👨‍👨‍👦🔫🍝"
    },
    {
        "movie": "타이타닉",
        "answer": "🚢❄️💔"
    },
    {
        "movie": "라라랜드",
        "answer": "🦸‍♂️🦸‍♀️⚡",
    },
    {
        "movie": "조커",
        "answer": "🎭🃏🪓",
    },
    {
        "movie": "얼음왕국",
        "answer": "❄️👸⛄"
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{movie}'),
    ('ai', '{answer}')
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)
final_prompt = ChatPromptTemplate.from_messages([

    ('system', """
    You're emoji generating AI bot base on movie.
    Generate three emojis that is representing it.
    Don't reply anything but three emojis.
    """),
    few_shot_prompt,
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{ask}',)
])
# chain = final_prompt | llm
# chain.invoke({'movie': '리틀 포레스트'})

chain = { "ask": RunnablePassthrough(), 'movie': RunnablePassthrough(),'chat_history': RunnableLambda(load_memory)} | final_prompt | llm


In [4]:
def ask_movie_for_emoji(movie: str):
    ask = f'What do you know about {movie}?. If I asked before, let me know.'
    result = chain.invoke({"ask": ask, 'movie': movie})
    chat_memory.save_context(
        inputs={'human': movie},
        outputs={'ai': result.content}
    )
    print(result.content)

In [5]:
ask_movie_for_emoji('The little mermaid')

🧜‍♀️🌊🐚


In [6]:
ask_movie_for_emoji('Forest gump')


🍫🏃‍♂️🌳


In [7]:
result = chain.invoke({'ask': '마지막으로 질문한 영화에 대해 이모지로 표현해줘?'})
print(result.content)

🍫🏃‍♂️🌳
